### Libgen bulk downloader

In [17]:
import wget
import urllib.request
import socket
import bardapi
from libgen.libgen_search import LibgenSearch
import subprocess
import re
import libgen
print(libgen.__file__)
from bardapi import Bard
import os
from dotenv import load_dotenv
load_dotenv()
BARD_TOKEN = os.getenv('BARD_TOKEN')

/Users/jm/Repos/ai-book-downloader/libgen/__init__.py


### BARD

In [ ]:
# USER INPUTS
topic = 'Macroeconomics'
limit = 5

In [ ]:
#bard = Bard(token_from_browser=True)
#res = bard.get_answer("Do you like cookies?")
#print(res)

In [ ]:
input_text = f"Create a list of the best {limit} books related to {topic}. Return your response as a python dictionary, with the author name as the key and book title as a value. Your answer has to contain ONLY the python dictionary!"
#bard_response = bard.get_answer(input_text)['content']
response = bardapi.core.Bard(BARD_TOKEN).get_answer(input_text)

In [ ]:
response['code']

In [ ]:
# Extract dict from response string
pattern = r"\{(.*?)\}"
match = re.search(pattern, response['code'], re.DOTALL)
if match:
    extracted_content = match.group(1)
    final_str = "{" + str(extracted_content) + "}"
    books_dict = eval(final_str)
    print(books_dict)

### LIBGEN

In [3]:
dummy_data = {'Mankiw': 'Principles of Economics', 'Romer': 'Advanced Macroeconomics', 'Krugman': 'Macroeconomics', 'Blanchard': 'Macroeconomics', 'Stiglitz': 'The Roaring Nineties'}

In [4]:
ls = LibgenSearch()

In [33]:
# DOWNLOAD FILE USING MIRRORS

filters = {"Language": "English", "Extension": "pdf"}
timeout = 240

metadata_list = []
for k,v in dummy_data.items():
    search_prompt = k+' '+v
    filename = k+'_'+v
    book_search = ls.search_holistic_filtered(f"{search_prompt}", filters, exact_match=False)
    if book_search:
        metadata_list.append(book_search[0])
        download_links = ls.resolve_download_links(book_search[0])
        for mirror, url in download_links.items():
            try:
                response = urllib.request.urlopen(url, timeout=timeout)
                content = response.read()
                save_path = f'downloads/{filename}.pdf'
                with open(save_path, 'wb') as file:
                    file.write(content)
                print(f'{mirror} file saved to {save_path}')
                break
            except urllib.error.URLError as e:
                print(f"Download from {mirror} mirror failed. Error: {e}")
            except socket.timeout:
                print(f"Download from {mirror} mirror timed out.")
else:
    print("Download from all mirrors failed.")

Download from GET mirror failed. Error: <urlopen error [Errno 60] Operation timed out>
Cloudflare file saved to downloads/Mankiw_Principles of Economics.pdf
Download from GET mirror failed. Error: <urlopen error [Errno 60] Operation timed out>
Cloudflare file saved to downloads/Romer_Advanced Macroeconomics.pdf
Download from GET mirror failed. Error: <urlopen error [Errno 60] Operation timed out>
Cloudflare file saved to downloads/Krugman_Macroeconomics.pdf
Download from GET mirror failed. Error: <urlopen error [Errno 51] Network is unreachable>
Cloudflare file saved to downloads/Blanchard_Macroeconomics.pdf
Download from all mirrors failed.
